# Dependencies


## Installation


In [ ]:
%pip install pip --upgrade -q
%pip install pandas -q
%pip install numpy -q
%pip install matplotlib -q
%pip install folium -q
%pip install geopy -q
%pip install geopandas -q

## Importing


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import geopandas as gpd

# Loading Data


## Loading Estimated Population Data


In [ ]:
POP_DATA_PATH = "data/population/tabela6579.csv" 

DTYPE = {"IBGE-code": str, "municipality-name": str, "population": int}

HEADER_SIZE = 3
FOOTER_SIZE = 151

pop_data = pd.read_csv(POP_DATA_PATH, sep=",", header= HEADER_SIZE, skipfooter=FOOTER_SIZE, engine='python', encoding="utf-8")
pop_data.columns = ["IBGE-code", "municipality-name", "population"]
pop_data = pop_data.astype(DTYPE)


In [ ]:
# show the first 5 rows of the data
pop_data.head(5)

In [ ]:
# show the last 5 rows of the data
pop_data.tail(5)

In [ ]:
# Removing the last digit of the IBGE code, wich is the check digit
pop_data["IBGE-code"] = pop_data["IBGE-code"].str[:-1]
# show the first 5 rows of the data 
pop_data.head(5)

## Loading Geo and Hospitalizations Data


In [ ]:
MUNICIPALITIES_WITH_REPORTS_FILE_PATH = "data/generated/mixed/municipalities_with_reports_jan2024-dec2024.csv"

DTYPE = {"IBGE-code": str}

# read the municipalities with reports data
municipalities_with_reports = pd.read_csv(MUNICIPALITIES_WITH_REPORTS_FILE_PATH, sep=";", encoding="utf-8", dtype=DTYPE)

In [ ]:
# show the first 5 rows of the data
municipalities_with_reports.head(5)

# Joining Data


In [ ]:
# Join the two dataframes on the IBGE code
RELEVANT_POP_DATA_COLUMNS = ["IBGE-code", "population"]
merged_data = municipalities_with_reports.merge(pop_data[RELEVANT_POP_DATA_COLUMNS], on="IBGE-code", how="inner")
# show the first 5 rows of the merged data
merged_data.head(5)

In [ ]:
# getting the reason between internacoes and population
merged_data["internacoes/population"] = merged_data["internacoes"] / merged_data["population"]
# show the first 5 rows of the merged data

In [ ]:
# saving the merged data to a csv file
MERGED_DATA_FILE_PATH = "data/generated/mixed/municipalities_with_reports_jan2024-dec2024_and_population.csv"
merged_data.to_csv(MERGED_DATA_FILE_PATH, sep=";", encoding="utf-8")

In [ ]:
# saving the merged data to a GeoJSON file
ESPG_FOR_FOLIUM= "EPSG:4326"
GEOJSON_FILE_PATH = "data/generated/mixed/municipalities_with_reports_jan2024-dec2024_and_population.geojson"
GEOJSON_FILE_PROPERTIES = ["IBGE-code", "IBGE-state-code", "IBGE-municipality-code", "municipality-name", "municipality-state", "internacoes", "population", "internacoes/population"]

geojson_file = gpd.GeoDataFrame(
    merged_data[GEOJSON_FILE_PROPERTIES], 
    geometry=gpd.GeoSeries.from_wkt(merged_data["geometry"]),
    crs=ESPG_FOR_FOLIUM
)

geojson_file.to_file(GEOJSON_FILE_PATH, driver="GeoJSON")

# Generating the Choromap


In [ ]:
BRAZILIA_COORDINATES = [-15.7801, -47.9292]
m = folium.Map(location=BRAZILIA_COORDINATES, zoom_start=4, control_scale=True, tiles="cartodb positron")


In [ ]:
# adding municipalities with no reports
MUNICIPALITIES_WITHOUT_REPORTS_FILE_PATH = "data/generated/mixed/municipalities_without_reports_jan2024-dec2024.csv"
municipalities_without_reports = pd.read_csv(
    MUNICIPALITIES_WITHOUT_REPORTS_FILE_PATH, 
    sep=";", 
    encoding="utf-8", 
    dtype=DTYPE
)

municipalities_without_reports = gpd.GeoDataFrame(
    municipalities_without_reports.drop(columns=["geometry"]),
    geometry=gpd.GeoSeries.from_wkt(municipalities_without_reports["geometry"]),
    crs=ESPG_FOR_FOLIUM
)

municipalities_without_reports.head(5)

folium.GeoJson(
    municipalities_without_reports,
    name="Municipalities without reports",
    style_function=lambda x: {
        "fillColor": "gray",
        "color": "black",
        "weight": 0.5,
        "fillOpacity": 0.5,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["IBGE-code", "municipality-name", "municipality-state"],
        aliases=["IBGE Code", "Municipality Name", "State"],
        localize=True,
    ),
).add_to(m)

In [ ]:
# adding the choropleth layer to the map
geo_data = gpd.read_file(GEOJSON_FILE_PATH)
geo_data_simplified = gpd.GeoDataFrame(
    data=geo_data.copy().drop(columns="geometry"),
    crs=geo_data.crs,
    geometry=geo_data["geometry"].copy().simplify(tolerance=0.1, preserve_topology=True)
)

folium.Choropleth(
    name="Hospitalizations per number of inhabitants",
    geo_data= geo_data,
    data = geo_data,
    columns=["IBGE-code", "internacoes/population"],
    key_on="feature.properties.IBGE-code",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Hospitalizations per number of inhabitants",
    highlight=True,
).add_to(m)

In [ ]:
# add transparent layer to show tooltip
folium.GeoJson(
    geo_data_simplified,
    name = "TRANSPARENT",
    control=True,
    show=True,
    overlay=True,
    popup="dfsdsdf",
    popup_keep_highlighted=False,
    style_function=lambda x: {
        "fillOpacity": 0,
        "color": "black",
        "weight": 0,

    },
    highlight_function=lambda x: {'weight': 3, 'color': '#00000000'},
    tooltip=folium.GeoJsonTooltip(
        fields=["IBGE-code", "IBGE-state-code", "IBGE-municipality-code", "municipality-state", "municipality-name", "internacoes"],
        aliases=["IBGE code", "IBGE state code", "IBGE municipality code", "Municipality state", "Municipality name" , "Internacoes"],
        localize=False,
        sticky=False,
        labels=True,
        style="background-color: white; color: black; font-family: Arial; font-size: 12px; padding: 5px;",
    ),
).add_to(m)

In [ ]:
# Preview the map
# m

In [ ]:
# Saving the map to an HTML file
MAP_FILE_PATH = "maps/cases_by_municipality_pop_jan2024-dez2024.html"
m.save(MAP_FILE_PATH)